<a href="https://www.kaggle.com/code/sonujha090/image-classification-indoor-scence?scriptVersionId=110406456" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# importing necessary libraries
import os 
from pathlib import Path
from glob import glob
import random

# Path to the dataset
path = Path('../input/indoor-scenes-cvpr-2019')
image_path = path/'indoorCVPR_09/Images' # Path to the images
image_files = glob(str(image_path/'**/*.jpg')) # List of all the images

# shuffle images
random.shuffle(image_files)

# label encoding for the classes
class_names = [p.name for p in image_path.glob('*') if p.is_dir()]

label2index = {v:k for k,v in enumerate(class_names)}
index2label = {v:k for k,v in label2index.items()}

# Creating a dataframe with the image path and the corresponding label
import pandas as pd
df = pd.DataFrame({'image':image_files})
df['label'] = df['image'].map(lambda x: label2index[x.split('/')[-2]])

# Splitting the dataset into train and test
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])

# Saving the train and test dataframe
train_df.to_csv('/train.csv', index=False)
test_df.to_csv('/test.csv', index=False)


In [ ]:
from torchvision import transforms
import torch 

class config:
    EPOCHS = 10
    BATCH_SIZE = 32
    LEARNING_RATE = 0.001
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    LOAD_MODEL = False
    SAVE_MODEL = True
    CHECKPOINT_FILE = "my_checkpoint.pth.tar"

    tfms = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ])

    # save checkpoint
    def save_checkpoint(state, filename="my_checkpoint.pth.tar"):
        print("=> Saving checkpoint")
        torch.save(state, filename)

    # load checkpoint
    def load_checkpoint(checkpoint, model):
        print("=> Loading checkpoint")
        model.load_state_dict(checkpoint['state_dict'])

    # Learning rate scheduler
    def lr_scheduler(optimizer, epoch, init_lr=0.001, lr_decay_epoch=5):
        """Decay learning rate by a factor of 0.1 every lr_decay_epoch epochs."""
        lr = init_lr * (0.1**(epoch // lr_decay_epoch))
        if epoch % lr_decay_epoch == 0:
            print('LR is set to {}'.format(lr))

        for param_group in optimizer.param_groups:
            param_group['lr'] = lr

        return optimizer  

In [27]:
from torch.utils.data import Dataset, DataLoader, Subset 
import pandas as pd 
import torch, torchvision
from PIL import Image 
from torchvision import transforms 

class IndoorSceneDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform
    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img = Image.open(row['image']).convert('RGB')
        if self.transform:
            img = self.transform(img)
        return img, row['label']

if __name__=='__main__':
    df = pd.read_csv('/train.csv')
    ds = IndoorSceneDataset(df, transform=tfms)
    dl = DataLoader(ds, batch_size=32, shuffle=True)
    for xb, yb in dl:
        print(xb.shape, yb.shape)
        break

torch.Size([32, 3, 224, 224]) torch.Size([32])


In [29]:
from tqdm import tqdm
import numpy as np 
import torch
from sklearn.metrics import accuracy_score

def train_model(model, train_dl, valid_dl, optimizer, criterion, num_epochs=10, device=device):
    train_losses, valid_losses, train_acc, valid_acc = [], [], [], []
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        train_correct = 0
        for x, y in tqdm(train_dl):
            optimizer.zero_grad()
            x, y = x.to(device), y.to(device)
            y_pred = model(x)
            loss = criterion(y_pred, y)
            loss.backward()
            optimizer.step()
            train_losses.append(loss.item())
            accuracy = accuracy_score(y.detach().cpu().numpy(), y_pred.argmax(dim=1).detach().cpu().numpy())
            train_acc.append(accuracy)
        model.eval()
        for x, y in tqdm(valid_dl):
            x, y = x.to(device), y.to(device)
            y_pred = model(x)
            loss = criterion(y_pred, y)
            accuracy = accuracy_score(y.detach().cpu().numpy(), y_pred.argmax(dim=1).detach().cpu().numpy())
            valid_losses.append(loss.item())
            valid_acc.append(accuracy)
        print(f'Epoch: {epoch+1}, Train Loss: {np.mean(train_losses):.4f}, Train Accuracy: {np.mean(train_acc):.4f}, Valid Loss: {np.mean(valid_losses):.4f}, Valid Accuracy: {np.mean(valid_acc):.4f}')

In [ ]:
%%time 

import torch, torchvision
from torch.utils.data import Dataset, DataLoader, Subset 
# from dataset import IndoorSceneDataset, tfms
import pandas as pd
import sys 


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
bs = 32

train_df = pd.read_csv('/train.csv')
valid_df = pd.read_csv('/test.csv')

train_dataset = IndoorSceneDataset(train_df, transform=tfms)
valid_dataset = IndoorSceneDataset(valid_df, transform=tfms)

train_dataset = Subset(train_dataset, range(0, int(len(train_dataset)*0.1)))
valid_dataset = Subset(train_dataset, range(0, int(len(valid_dataset)*0.1)))

train_dl = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_dl = DataLoader(valid_dataset, batch_size=32, shuffle=True)


model = torchvision.models.resnet18(pretrained=True)
for param in model.parameters():
    param.requires_grad = False
model.fc = torch.nn.Linear(512, 67)

# load checkpoint
if config.LOAD_MODEL and config.CHECKPOINT_FILE in os.listdir():
    load_checkpoint(torch.load(config.CHECKPOINT_FILE), model)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

model = model.to(device) 

train_model(model, train_dl, valid_dl, optimizer, criterion, num_epochs=5)

# save checkpoint
if config.SAVE_MODEL:
    checkpoint = {
        'state_dict': model.state_dict(),
        'optimizer': optimizer.state_dict()
    }
    save_checkpoint(checkpoint)

=> Loading checkpoint


 80%|████████  | 32/40 [00:12<00:03,  2.02it/s]

In [42]:
# # load checkpoint
# if config.LOAD_MODEL and config.CHECKPOINT_FILE in os.listdir():
#     load_checkpoint(torch.load(config.CHECKPOINT_FILE), model)

# model = model.to(device) 

# train_model(model, train_dl, valid_dl, optimizer, criterion, num_epochs=5)

# # save checkpoint
# if config.SAVE_MODEL:
#     checkpoint = {
#         'state_dict': model.state_dict(),
#         'optimizer': optimizer.state_dict()
#     }
#     save_checkpoint(checkpoint)

=> Loading checkpoint


 65%|██████▌   | 26/40 [00:10<00:05,  2.48it/s]


KeyboardInterrupt: 